In [1]:
# Required libraries

import os
import glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
path_weights = '../data/weights/'

# Importing ppi_tf_merge_cluster_ges

In [3]:
df_ppi_tf = pd.read_csv('../third_party/PMC5737331/NN_code_release/ppi_tf_merge_cluster.txt', sep=' ', header=None)
df_ppi_tf.shape

(696, 2)

In [4]:
df_ppi_tf['ppi_or_tf'] = 'ppi'
df_ppi_tf.loc[(df_ppi_tf.loc[df_ppi_tf[0]=='TF:'].index), 'ppi_or_tf'] = 'tf'
df_ppi_tf.groupby('ppi_or_tf').size()

ppi_or_tf
ppi    348
tf     348
dtype: int64

In [5]:
# WEIGHT TABLE for PPI and TF
df_tf = df_ppi_tf.loc[df_ppi_tf['ppi_or_tf'] == 'tf'][[1]].reset_index(drop=True)
df_tf = df_tf.rename(columns={1:'gene'})
df_ppi = df_ppi_tf.loc[df_ppi_tf['ppi_or_tf'] == 'ppi'][[0]].reset_index(drop=True)
df_ppi = df_ppi.rename(columns={0:'gene'})
## removing 'ppi_xx\t' prefix from dataset
df_ppi = df_ppi.replace(regex='ppi_\d{1,4}\\t', value='')

In [6]:
# Exporting the gene list which shows the details about genes and their belonging nodes
list_tf = [item for sublist in df_tf['gene'].str.split('\t') for item in sublist]
list_ppi = [item for sublist in df_ppi['gene'].str.split('\t') for item in sublist]

In [7]:
print('lenght of list_tf  (all)   , ', len(list_tf))
print('lenght of list_ppi (all)   , ', len(list_ppi))
print('')
print('lenght of list_tf  (unique), ', len(np.unique(list_tf)))
print('lenght of list_ppi (unique), ', len(np.unique(list_ppi)))
print('')
list_ppi_tf_union =  list(set(list_ppi).union(set(list_tf)))
print('lenght of list_common_gene , ', len(list_ppi_tf_union))

lenght of list_tf  (all)   ,  51012
lenght of list_ppi (all)   ,  5885

lenght of list_tf  (unique),  8307
lenght of list_ppi (unique),  3553

lenght of list_common_gene ,  9520


In [8]:
df_tf['node'] = pd.DataFrame(['tf_'+str(x) for x in df_tf.index.values])
df_ppi['node'] = pd.DataFrame(['ppi_'+str(x) for x in df_ppi.index.values])

# Creating weight table for TF nodes

In [9]:
df_tf_weight = pd.DataFrame(np.unique(list_tf))
df_tf_weight = df_tf_weight.rename(columns={0:'symbol'})

for i, node_ in enumerate(df_tf['node']):
#    print(i, node_)
    df_tf_weight[node_] = 0.0
    list_isin = df_tf.iloc[i]['gene'].split('\t')
    df_tf_weight.loc[df_tf_weight['symbol'].isin(list_isin), node_] = 1.0

In [10]:
# Exporting TF weight table
df_tf_weight.to_csv(os.path.join(path_weights + 'tf_weight.txt'), index=False)

# Creating weight table for TF nodes

In [11]:
df_ppi_weight = pd.DataFrame(np.unique(list_ppi))
df_ppi_weight = df_ppi_weight.rename(columns={0:'symbol'})

for i, node_ in enumerate(df_ppi['node']):
#    print(i, node_)
    df_ppi_weight[node_] = 0.0
    list_isin = df_ppi.iloc[i]['gene'].split('\t')
    df_ppi_weight.loc[df_ppi_weight['symbol'].isin(list_isin), node_] = 1.0

In [12]:
# Exporting TF weight table
df_ppi_weight.to_csv(os.path.join(path_weights + 'ppi_weight.txt'), index=False)